In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as ssp

In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    return ssp.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                          shape = loader['shape'])

In [5]:
# dtm = load_sparse_csr('small_dtm.npz')
with open('red_dtm_vocabulary.txt', 'r') as vocfile:
    vocabulary = [x.strip('\n') for x in vocfile.readlines()]

In [5]:
df = dtm.toarray()
df1 = pd.DataFrame(df)
del dtm,df

In [6]:
df_y = pd.read_csv('reports_metadata.csv')

In [5]:
df_y.columns

Index(['file_name', 'new_filename', 'country_iso3c', 'country_name',
       'report_name', 'year.0', 'organization', 'word_count', 'hathaway',
       'state', 'fariss.mean', 'fariss.std_deviation', 'country_cow_code',
       'CIRI_codings.empowerment.electoral_rights',
       'CIRI_codings.empowerment.assembly',
       'CIRI_codings.empowerment.domestic_movement',
       'CIRI_codings.empowerment.religious_rights',
       'CIRI_codings.empowerment.speech',
       'CIRI_codings.empowerment.foreign_movement',
       'CIRI_codings.empowerment.worker_rights',
       'CIRI_codings.physical_integrity.dissapearance',
       'CIRI_codings.physical_integrity.imprisonment',
       'CIRI_codings.physical_integrity.torture',
       'CIRI_codings.physical_integrity.killing', 'genocide', 'amnesty'],
      dtype='object')

In [7]:
y1 = df_y['CIRI_codings.empowerment.domestic_movement'] # hathaway
# df1['y'] = y1
# df1

In [9]:
y1=y1.dropna()

In [8]:
X = pd.read_csv("small_dtm.csv")

In [26]:
X

,new_filename,Unnamed: 1,continu,death,group,sentenc,their,prison,peopl,who,...,may,offic,kill,use,children,employ,abus,labor,0,percent
0,ALB_1974_Amnesty_International.txt,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,DZA_1974_Amnesty_International.txt,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AGO_1974_Amnesty_International.txt,0,1,0,0,0,0,4,0,1,...,0,0,0,0,0,0,0,0,0,0
3,ARG_1974_Amnesty_International.txt,2,0,4,4,0,3,1,0,2,...,0,0,0,0,0,0,0,0,0,0
4,BHS_1974_Amnesty_International.txt,0,0,1,0,1,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14185,VNM_2013_State_Department.txt,11,94,5,26,33,68,63,29,32,...,47,13,3,27,52,25,30,41,0,23
14186,ESH_2013_State_Department.txt,1,6,1,10,3,9,25,1,10,...,2,8,1,9,2,5,18,11,0,5
14187,YEM_2013_State_Department.txt,1,28,17,54,6,57,59,3,26,...,19,30,29,30,43,17,30,44,0,29
14188,ZMB_2013_State_Department.txt,9,16,8,28,12,42,84,4,34,...,24,32,8,19,51,27,33,56,0,27


In [25]:
df_y

,file_name,new_filename,country_iso3c,country_name,report_name,year.0,organization,word_count,hathaway,state,...,CIRI_codings.empowerment.religious_rights,CIRI_codings.empowerment.speech,CIRI_codings.empowerment.foreign_movement,CIRI_codings.empowerment.worker_rights,CIRI_codings.physical_integrity.dissapearance,CIRI_codings.physical_integrity.imprisonment,CIRI_codings.physical_integrity.torture,CIRI_codings.physical_integrity.killing,genocide,amnesty
0,hwr1993_EGYPT.txt,EGY_1993_Human_Rights_Watch.txt,EGY,EGYPT,egypt,1993,Human Rights Watch,3403,4.0,4.0,...,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,4.0
1,hwr1993_HUMAN RIGHTS WATCH AND CONGRESSIONAL C...,human_rights_watch_and_congressional_casework_...,Not resolved,Not resolved,human_rights_watch_and_congressional_casework,1993,Human Rights Watch,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hwr1993_TUNISIA.txt,TUN_1993_Human_Rights_Watch.txt,TUN,TUNISIA,tunisia,1993,Human Rights Watch,2387,3.0,3.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,3.0
3,hwr1993_MOZAMBIQUE.txt,MOZ_1993_Human_Rights_Watch.txt,MOZ,MOZAMBIQUE,mozambique,1993,Human Rights Watch,1980,3.0,4.0,...,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,0.0,3.0
4,hwr1993_CAMBODIA.txt,KHM_1993_Human_Rights_Watch.txt,KHM,CAMBODIA,cambodia,1993,Human Rights Watch,2694,3.0,4.0,...,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14185,AI_Report_1986.txt,ai_report_1986_1985_Amnesty_International.txt,Not resolved,Not resolved,ai_report_1986,1985,Amnesty International,150596,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14186,AI_Report_1996.txt,ai_report_1996_1995_Amnesty_International.txt,Not resolved,Not resolved,ai_report_1996,1995,Amnesty International,204291,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14187,AI_Report_1998.txt,ai_report_1998_1997_Amnesty_International.txt,Not resolved,Not resolved,ai_report_1998,1997,Amnesty International,222911,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14188,AI_Report_1993.txt,ai_report_1993_1992_Amnesty_International.txt,Not resolved,Not resolved,ai_report_1993,1992,Amnesty International,207710,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
new_df = pd.merge(X,df_y,on='new_filename')
new_df.shape

(14546, 126)

In [33]:
new_df.drop(new_df.columns[0],axis=1,inplace=True)
new_df.drop(new_df.columns[0],axis=1,inplace=True)
new_df

,continu,death,group,sentenc,their,prison,peopl,who,releas,such,...,CIRI_codings.empowerment.religious_rights,CIRI_codings.empowerment.speech,CIRI_codings.empowerment.foreign_movement,CIRI_codings.empowerment.worker_rights,CIRI_codings.physical_integrity.dissapearance,CIRI_codings.physical_integrity.imprisonment,CIRI_codings.physical_integrity.torture,CIRI_codings.physical_integrity.killing,genocide,amnesty
0,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,0,0,0,0,1,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,1,0,0,0,0,4,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,4,4,0,3,1,0,2,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,0,1,0,1,0,0,0,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14541,94,5,26,33,68,63,29,32,15,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
14542,6,1,10,3,9,25,1,10,3,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14543,28,17,54,6,57,59,3,26,13,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
14544,16,8,28,12,42,84,4,34,10,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [34]:
from sklearn.model_selection import train_test_split
df1 = new_df.iloc[:,:99]
y=new_df['CIRI_codings.physical_integrity.torture']

In [35]:
df1['y'] = y
df1.dropna(inplace=True)
df1.shape

(10746, 100)

In [15]:
def create_baseline(X_train,y_train,n_epoch=10,patience=10):
    model = Sequential()
    model.add(Dense(1000, input_dim=X_train.shape[1], activation='relu')) # must be same to input columns
    model.add(Dense(1000)) # New hidden layer
    model.add(Dense(1000))
    model.add(Dense(1, activation='sigmoid')) # must be same to number of classes if hot encoding
    callbacks=[EarlyStopping(monitor='val_loss',patience=patience, min_delta=0.001)]
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.fit(X_train, y_train, epochs=n_epoch, validation_split=0.2, batch_size=200, verbose=1, callbacks = callbacks)
    return model

In [36]:
df_train, df_test = train_test_split(df1, test_size=0.3)
X_train, y_train = df_train.loc[:, df_train.columns != 'y'], df_train['y']
X_test, y_test = df_test.loc[:, df_test.columns != 'y'], df_test['y']

In [37]:
import time
start = time.time()
model = create_baseline(X_train, y_train,n_epoch=100)
end = time.time()
print("Runtime:",end - start)
yhat_classes = model.predict_classes(X_test.values, verbose=1)
yhat_classes = yhat_classes[:, 0]
accuracy = accuracy_score(y_test, yhat_classes)
print('***********Accuracy: %f' % accuracy)

Train on 6017 samples, validate on 1505 samples
Epoch 1/100
6017/6017 [==============================] - 2s 405us/step - loss: 0.9249 - mse: 0.9249 - val_loss: 0.9271 - val_mse: 0.9271
Epoch 2/100
6017/6017 [==============================] - 2s 396us/step - loss: 0.9366 - mse: 0.9366 - val_loss: 0.9270 - val_mse: 0.9270
Epoch 3/100
6017/6017 [==============================] - 2s 408us/step - loss: 0.9365 - mse: 0.9365 - val_loss: 0.9269 - val_mse: 0.9269
Epoch 4/100
6017/6017 [==============================] - 3s 452us/step - loss: 0.9365 - mse: 0.9365 - val_loss: 0.9269 - val_mse: 0.9269
Epoch 5/100
6017/6017 [==============================] - 2s 415us/step - loss: 0.9365 - mse: 0.9365 - val_loss: 0.9269 - val_mse: 0.9269
Epoch 6/100
6017/6017 [==============================] - 2s 394us/step - loss: 0.9365 - mse: 0.9365 - val_loss: 0.9269 - val_mse: 0.9269
Epoch 7/100
6017/6017 [==============================] - 3s 418us/step - loss: 0.9365 - mse: 0.9365 - val_loss: 0.9269 - val_mse: 

In [38]:
from sklearn.metrics import mean_squared_error as mse
err = mse(y_test,yhat_classes)
print(err)

0.9652605459057072


In [39]:
from sklearn.metrics import mean_squared_error as mse
clf = DecisionTreeRegressor()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
err = mse(y_test,y_pred)
print(err)

0.8466535704438931


In [40]:
from sklearn.ensemble import RandomForestRegressor
clf1 = RandomForestRegressor()
clf1.fit(X_train,y_train)
y_pred = clf1.predict(X_test)
err = mse(y_test,y_pred)
print(err)

0.4169120250830998


In [41]:
from sklearn.svm import SVR
clf2 = SVR()
clf2.fit(X_train,y_train)
y_pred = clf2.predict(X_test)
err = mse(y_test,y_pred)
print(err)

0.5134574433856961
